# Projeto de Eficiência Energética - UCI Energy Efficiency Dataset

# **Plano para AT2 - Modelagem e Avaliação de Algoritmos**

**por Tiago Dutra Galvão**


# **1. Preparação dos Dados** 

**Primeiro, vamos realizar o pré-processamento dos dados, aplicando as técnicas recomendadas em nossa análise exploratória:**

In [2]:
# Notebook: at2_modelagem/01_preprocessamento.ipynb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados processados da etapa AT1
df = pd.read_csv('../../data/processed/energy_efficiency_processed.csv')

# Exibir primeiras linhas
print("Primeiras linhas do dataset:")
display(df.head())

# Verificar tipos de dados
print("\nTipos de dados:")
display(df.dtypes)

# Separar features e alvos
X = df.drop(['Heating_Load', 'Cooling_Load'], axis=1)
y_heating = df['Heating_Load']
y_cooling = df['Cooling_Load']

# Identificar colunas categóricas e numéricas
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Colunas categóricas: {categorical_cols}")
print(f"Colunas numéricas: {numerical_cols}")

# Dividir os dados em conjuntos de treino e teste (70% treino, 30% teste)
X_train, X_test, y_heating_train, y_heating_test, y_cooling_train, y_cooling_test = train_test_split(
    X, y_heating, y_cooling, test_size=0.3, random_state=42
)

print(f"Tamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")

# Criar transformador de colunas para pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ],
    remainder='passthrough'
)

# Aplicar o pré-processamento aos conjuntos de treino e teste
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Se a transformação resultar em uma matriz esparsa, convertê-la para array
if hasattr(X_train_preprocessed, "toarray"):
    X_train_preprocessed = X_train_preprocessed.toarray()
if hasattr(X_test_preprocessed, "toarray"):
    X_test_preprocessed = X_test_preprocessed.toarray()

# Obter os nomes das colunas após one-hot encoding
if categorical_cols:
    cat_encoder = preprocessor.named_transformers_['cat']
    cat_cols_after_encoding = cat_encoder.get_feature_names_out(categorical_cols)
    feature_names = list(numerical_cols) + list(cat_cols_after_encoding)
else:
    feature_names = numerical_cols

print(f"Features após pré-processamento: {feature_names}")
print(f"Dimensões dos dados pré-processados: {X_train_preprocessed.shape}")

# Salvar os dados pré-processados
np.save('../../data/processed/X_train_preprocessed.npy', X_train_preprocessed)
np.save('../../data/processed/X_test_preprocessed.npy', X_test_preprocessed)
np.save('../../data/processed/y_heating_train.npy', y_heating_train)
np.save('../../data/processed/y_heating_test.npy', y_heating_test)
np.save('../../data/processed/y_cooling_train.npy', y_cooling_train)
np.save('../../data/processed/y_cooling_test.npy', y_cooling_test)

# Salvar o preprocessador para uso posterior
import joblib
joblib.dump(preprocessor, '../../models/preprocessor.pkl')

print("Pré-processamento concluído e dados salvos com sucesso.")

Primeiras linhas do dataset:


,Relative_Compactness,Surface_Area,Wall_Area,Roof_Area,Overall_Height,Orientation,Glazing_Area,Glazing_Area_Distribution,Heating_Load,Cooling_Load
0,0.98,514.5,294.0,110.25,7.0,Norte,0.0,Sem Vidraças,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,Leste,0.0,Sem Vidraças,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,Sul,0.0,Sem Vidraças,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,Oeste,0.0,Sem Vidraças,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,Norte,0.0,Sem Vidraças,20.84,28.28



Tipos de dados:


Relative_Compactness         float64
Surface_Area                 float64
Wall_Area                    float64
Roof_Area                    float64
Overall_Height               float64
Orientation                   object
Glazing_Area                 float64
Glazing_Area_Distribution     object
Heating_Load                 float64
Cooling_Load                 float64
dtype: object

Colunas categóricas: ['Orientation', 'Glazing_Area_Distribution']
Colunas numéricas: ['Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area', 'Overall_Height', 'Glazing_Area']
Tamanho do conjunto de treino: (537, 8)
Tamanho do conjunto de teste: (231, 8)
Features após pré-processamento: ['Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area', 'Overall_Height', 'Glazing_Area', 'Orientation_Norte', 'Orientation_Oeste', 'Orientation_Sul', 'Glazing_Area_Distribution_Norte', 'Glazing_Area_Distribution_Oeste', 'Glazing_Area_Distribution_Sem Vidraças', 'Glazing_Area_Distribution_Sul', 'Glazing_Area_Distribution_Uniforme']
Dimensões dos dados pré-processados: (537, 14)
Pré-processamento concluído e dados salvos com sucesso.


**Próximo passso será gerar os modelos de regressão no arquivo notebook at2_modelagem/02_modelagem.ipynb para implementar os cinco modelos de regressão:**